<a href="https://colab.research.google.com/github/ArjunNarayan2066/CS484_project/blob/fixed_training_loop/cs484_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install dependencies
! nvcc --version

! pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
! pip install tensorboardX==1.4
! pip install opencv-python==3.3.1.11

# Clone repo
! git clone https://github.com/ArjunNarayan2066/monodepth2.git

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4MB 24kB/s 
     |████████████████████████████████| 12.8MB 242kB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


     |████████████████████████████████| 71kB 5.0MB/s 
     |████████████████████████████████| 24.7MB 129kB/s 
ERROR: dopamine-rl 1.0.5 has requirement opencv-python>=3.4.1.15, but you'll have opencv-python 3.3.1.11 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
Cloning into 'monodepth2'...
remote: Enumerating objects: 148, done.
remote: Total 148 (delta 0), reused 0 (delta 0), pack-reused 148
Receiving objects: 100% (148/148), 10.26 MiB | 37.38 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [ ]:
# ! python monodepth2/test_simple.py --image_path monodepth2/assets/test_image.jpg --model_name mono+stereo_640x192 --model_path /root/tmp/S_640x192/models/weights_19
# /root/tmp/S_640x192/models/weights_9/encoder.pth

In [1]:
import torch
! uname -a
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

Linux 28dd3f85ea1d 4.19.112+ #1 SMP Thu Jul 23 08:00:38 PDT 2020 x86_64 x86_64 x86_64 GNU/Linux
True
Tesla T4


In [ ]:
# ! python monodepth2/train.py --model_name stereo_model  --frame_ids 0 --use_stereo --split eigen_full

In [3]:
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_28_calib.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_28_drive_0001/2011_09_28_drive_0001_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_28_drive_0002/2011_09_28_drive_0002_sync.zip



--2020-12-18 04:00:17--  https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_28_calib.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.74.199
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.74.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [application/zip]
Saving to: ‘2011_09_28_calib.zip’

2011_09_28_calib.zi 100%[===================>]   3.98K  --.-KB/s    in 0s      

2020-12-18 04:00:18 (182 MB/s) - ‘2011_09_28_calib.zip’ saved [4073/4073]

--2020-12-18 04:00:18--  https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_28_drive_0001/2011_09_28_drive_0001_sync.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.74.199
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.74.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 424412944 (405M) [application/zip]


In [4]:
! mkdir kitti_data
! unzip -q 2011_09_28_drive_0001_sync.zip -d kitti_data
! rm -rf 2011_09_28_drive_0001_sync.zip
! unzip -q 2011_09_28_drive_0002_sync.zip -d kitti_data
! rm -rf 2011_09_28_drive_0002_sync.zip
! unzip -q 2011_09_28_calib.zip -d kitti_data
! rm -rf 2011_09_28_calib.zip

In [7]:
! sudo apt update
! sudo apt install imagemagick
! sudo apt install parallel
! find kitti_data/2011_09_28 -name '*.png' | parallel 'convert -quality 92 -sampling-factor 2x2,1x1,1x1 {.}.png {.}.jpg && rm {}'

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [ ]:
# ! python monodepth2/train.py --model_name S_640x192 --frame_ids 0 --use_stereo --pose_model_type separate_resnet --split eigen_full --data_path /content/kitti_data --num_epochs 10
# /content/kitti_data/2011_09_26/2011_09_26_drive_0106_sync/image_02/data/0000000115.png

False


In [ ]:
# ! python monodepth2/export_gt_depth.py --data_path kitti_data --split eigen

In [ ]:
# ! python monodepth2/evaluate_depth.py --data_path kitti_data --load_weights_folder /root/tmp/S_640x192/models/weights_19/ --eval_stereo

In [1]:
#@title Default title text
### For Training
import monodepth2.layers, monodepth2.utils, monodepth2.trainer
import monodepth2.networks

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import timeit
import PIL.Image as pil


class MyTraining(object):
    def __init__(self, batch):
        self.batch = batch
        self.h = 192
        self.w = 640
        if torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")

        # For listing which frames to check
        # For simple stereo is only current (0) and estimated pair (s)
        self.frames = [0, 's']

        # For multi-scale estimation
        self.loss_scales = [0, 1, 2, 3]

        # Weighting for loss function
        self.alpha = 0.85

        # Define projections for pose estimation
        self.depth_projection = {}
        self.projection_3d = {}
        for loss_scale in self.loss_scales:
            h = int(self.h / (2 ** loss_scale))
            w = int(self.w / (2 ** loss_scale))

            self.depth_projection[loss_scale] = monodepth2.layers.BackprojectDepth(self.batch, h, w).to(self.device)
            self.projection_3d[loss_scale] = monodepth2.layers.Project3D(self.batch, h, w).to(self.device)
        
        ## Declare Depth Network
        self.depth_encoder_network = monodepth2.networks.ResnetEncoder(18, True).to(self.device)
        self.depth_decoder_network = monodepth2.networks.DepthDecoder(self.depth_encoder_network.num_ch_enc, 
                                                [0, 1, 2, 3]).to(self.device) 

        ## Declare Pose Network
        self.pose_encoder_network = monodepth2.networks.ResnetEncoder(18, True, num_input_images=2).to(self.device)
        self.pose_decoder_network = monodepth2.networks.PoseDecoder(self.pose_encoder_network.num_ch_enc, 
                                                num_input_features=1, num_frames_to_predict_for=2).to(self.device)

        self.models = {"encoder": self.depth_encoder_network, "depth": self.depth_decoder_network,
                       "pose_encoder": self.pose_encoder_network, "pose": self.pose_decoder_network}

        # Set up our excerpt of Kitti Dataset
        training_files = monodepth2.utils.readlines("/content/monodepth2/splits/eigen_full/train_files.txt")
        validation_files = monodepth2.utils.readlines("/content/monodepth2/splits/eigen_full/val_files.txt")

        train_set = monodepth2.datasets.kitti_dataset.KITTIRAWDataset("/content/kitti_data", training_files, self.h, self.w, self.frames, 4, is_train=True, img_ext='.jpg')
        self.train_loader = DataLoader(train_set, self.batch, True, num_workers=6, pin_memory=True, drop_last=True)
        val_set = monodepth2.datasets.kitti_dataset.KITTIRAWDataset("/content/kitti_data", validation_files, self.h, self.w, self.frames, 4, is_train=False, img_ext='.jpg')
        self.val_loader = DataLoader(val_set, self.batch, True, num_workers=6, pin_memory=True, drop_last=True)

    def run_training_loop(self):
        self.all_params = list(self.depth_encoder_network.parameters())
        self.all_params += list(self.depth_decoder_network.parameters())
        self.all_params += list(self.pose_encoder_network.parameters())
        self.all_params += list(self.pose_decoder_network.parameters())

        # Same learning rate configuration as https://arxiv.org/pdf/1806.01260.pdf
        self.adam_optim = optim.Adam(self.all_params, 1e-4)
        self.lr_sched = optim.lr_scheduler.StepLR(self.adam_optim, 15, 0.1)
        self.ssim_loss_func = monodepth2.layers.SSIM().to(self.device)

        self.epoch_count = 0
        losses  = []

        for self.epoch_count in range(2):
            # Per Epoch
            self.lr_sched.step()
            self.depth_encoder_network.train()
            self.depth_decoder_network.train()
            self.pose_encoder_network.train()
            self.pose_decoder_network.train()

            for idx, inputs in enumerate(self.train_loader):
                # Push to GPU
                for key, ipt in inputs.items():
                    inputs[key] = ipt.to(self.device)

                # Per Batch Code
                batch_start_time = timeit.default_timer()

                feature_identifications = self.depth_encoder_network(inputs["color_aug", 0, 0])
                outputs = self.depth_decoder_network(feature_identifications)

                self.estimate_stereo_predictions(inputs, outputs)
                loss = self.batch_loss_func(inputs, outputs)

                self.adam_optim.zero_grad()
                loss["loss"].backward()
                losses.append(loss["loss"].item())
                self.adam_optim.step()

                batch_duration = timeit.default_timer() - batch_start_time

                # Do something with batch duration and losses

    def estimate_stereo_predictions(self, inputs, outputs):
        # Generate estimated stereo pair using the pose networks
        for loss_scale in self.loss_scales:
            estimated_disparity = outputs[("disp", loss_scale)]
            disp = F.interpolate(estimated_disparity, [self.h, self.w], mode="bilinear", align_corners=False)
            base = 0 # base scale

            # Change names
            scaled_disp = 0.001 + (10 - 0.001) * disp
            depth = 1 / scaled_disp

            outputs[("depth", 0, loss_scale)] = depth

            # Run for stereo pair
            T = inputs["stereo_T"]

            camera_coords = self.depth_projection[base](depth, inputs[("inv_K", base)])
            pixel_coords = self.projection_3d[base](camera_coords, inputs[("K", base)], T)

            outputs[("sample", 's', loss_scale)] = pixel_coords
            outputs[("color", 's', loss_scale)] = F.grid_sample(
                inputs[("color", 's', base)], outputs[("sample", 's', loss_scale)], padding_mode="border")
                
    def reprojection(self, prediction, target):
        ssim_loss = self.ssim_loss_func(prediction, target).mean(1, True)
        reproj_losses = self.alpha*ssim_loss + (1-self.alpha)*(torch.abs(target-prediction).mean(1, True))
        return reproj_losses

    def batch_loss_func(self, inputs, outputs):
        batch_loss = {}
        complete_losses = 0

        for loss_scale in self.loss_scales:
            current_loss = 0

            base = 0

            reproj_losses = []

            reproj_losses.append(self.reprojection(outputs[("color", 's', loss_scale)], inputs[("color", 0, base)]))
            reproj_losses = torch.cat(reproj_losses, 1)

            identity_loss = []
            identity_loss.append(self.reprojection(inputs[("color", 's', base)], inputs[("color", 0, base)]))
            identity_loss = torch.cat(identity_loss, 1)


            # Add some minor noise to ensure no repeated values
            identity_loss += torch.rand(identity_loss.shape).cuda() * 0.00001
            total = torch.cat((identity_loss, reproj_losses), dim=1)
            val, idxs = torch.min(total, dim=1)

            # UNDERSTAND THIS LINE MORE
            outputs["identity_selection/{}".format(loss_scale)] = (idxs > identity_loss.shape[1] - 1).float()

            current_loss += val.mean()

            mean_disp = outputs[("disp", loss_scale)].mean(2, True).mean(3, True)
            norm_disp = outputs[("disp", loss_scale)] / (mean_disp + 1e-7)
            smooth_loss = monodepth2.layers.get_smooth_loss(norm_disp, inputs[("color", 0, loss_scale)])

            current_loss += 1e-3 * smooth_loss / (2 ** loss_scale)
            complete_losses += current_loss
            batch_loss["loss/{}".format(loss_scale)] = current_loss

        complete_losses /= len(self.loss_scales)
        batch_loss["loss"] = complete_losses
        return batch_loss


In [2]:

import os
import torchvision.models as models

def save_model(train):
    """Save model weights to disk
    """
    save_folder = os.path.join("/content/test_model/")
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    for model_name, model in train.models.items():
        save_path = os.path.join(save_folder, "{}.pth".format(model_name))
        to_save = model.state_dict()
        if 'encoder' in model_name:
            # save the sizes - these are needed at prediction time
            to_save['height'] = train.h
            to_save['width'] = train.w
            to_save['use_stereo'] = True
        torch.save(to_save, save_path)

    save_path = os.path.join(save_folder, "{}.pth".format("adam"))
    torch.save(train.adam_optim.state_dict(), save_path)

In [ ]:
train = MyTraining(6)
train.run_training_loop()
save_model(train)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


In [41]:
! python monodepth2/test_simple.py --image_path monodepth2/assets/test_image.jpg --model_name mono+stereo_640x192 --model_path /content/test_model/

# import PIL.Image as pil
# import matplotlib.pyplot as plt
# from torchvision import transforms

# image_path = "monodepth2/assets/test_image.jpg"

# input_image = pil.open(image_path).convert('RGB')
# original_width, original_height = input_image.size

# feed_height = train.h
# feed_width = train.w
# input_image_resized = input_image.resize((feed_width, feed_height), pil.LANCZOS)

# input_image_pytorch = transforms.ToTensor()(input_image_resized).unsqueeze(0)

# with torch.no_grad():
#     features = train.depth_decoder_network(input_image_pytorch)
#     outputs = train.depth_decoder_network(features)

# disp = outputs[("disp", 0)]

# disp_resized = torch.nn.functional.interpolate(disp,
#     (original_height, original_width), mode="bilinear", align_corners=False)

# # Saving colormapped depth image
# disp_resized_np = disp_resized.squeeze().cpu().numpy()
# vmax = np.percentile(disp_resized_np, 95)

# plt.figure(figsize=(10, 10))
# plt.subplot(211)
# plt.imshow(input_image)
# plt.title("Input", fontsize=22)
# plt.axis('off')

# plt.subplot(212)
# plt.imshow(disp_resized_np, cmap='magma', vmax=vmax)
# plt.title("Disparity prediction", fontsize=22)
# plt.axis('off')


-> Loading model from  /content/test_model/
   Loading pretrained encoder
   Loading pretrained decoder
-> Predicting on 1 test images
   Processed 1 of 1 images - saved prediction to monodepth2/assets/test_image_disp.jpeg
-> Done!
